In [2]:
%config Completer.use_jedi = False

In [164]:
import spacy
from spacy.attrs import *
nlp = spacy.load('en_core_web_lg')
from spacy import attrs
import pandas as pd

In [198]:
comment = 'The condition of the bike was poor but the service is easy on the wallet'
doc = nlp(comment)
data = sorted([(token.head.i, token.i, token, token.head, (token.head.pos_=='AUX'), (token.head.pos_==token.pos_ and not token.head.i == token.i)) for token in doc], key=lambda x: x[0], reverse=False)
df = pd.DataFrame(data, columns =['token head index', 'token index', 'token', 'token head', 'Is token AUX?', 'is token ready for cutoff?'])

In [199]:
df

,token head index,token index,token,token head,Is token AUX?,is token ready for cutoff?
0,1,0,The,condition,False,False
1,1,2,of,condition,False,False
2,2,4,bike,of,False,False
3,4,3,the,bike,False,False
4,5,1,condition,was,True,False
5,5,5,was,was,True,False
6,5,6,poor,was,True,False
7,5,7,but,was,True,False
8,5,10,is,was,True,True
9,9,8,the,service,False,False


In [278]:
comment = 'The bike was in poor condition but the service is easy on the wallet. Hyderabad is a city. Karthik is my name.'
doc = nlp(comment)
data = sorted([(token.head.i, token.i, token, token.head, (token.head.pos_=='AUX'), (token.head.pos_==token.pos_ and not token.head.i == token.i)) for token in doc], key=lambda x: x[0], reverse=False)
df = pd.DataFrame(data, columns =['token head index', 'token index', 'token', 'token head', 'Is token connected to AUX?', 'is token ready for cutoff?'])

In [279]:
df = df[df['is token ready for cutoff?'] == False]

In [383]:
title_list = ['is conjunct?']
for token in doc:
    print(token.tensor.shape, token.tag_, token.dep_, token.head.i, token.is_sent_start, token.lemma_, doc.ents, token.ent_iob_, token.ent_type_)

(96,) DT det 1 True the (Hyderabad,) O 
(96,) NN nsubj 2 False bike (Hyderabad,) O 
(96,) VBD ROOT 2 False be (Hyderabad,) O 
(96,) IN prep 2 False in (Hyderabad,) O 
(96,) JJ amod 5 False poor (Hyderabad,) O 
(96,) NN pobj 3 False condition (Hyderabad,) O 
(96,) CC cc 2 False but (Hyderabad,) O 
(96,) DT det 8 False the (Hyderabad,) O 
(96,) NN nsubj 9 False service (Hyderabad,) O 
(96,) VBZ conj 2 False be (Hyderabad,) O 
(96,) JJ acomp 9 False easy (Hyderabad,) O 
(96,) IN prep 9 False on (Hyderabad,) O 
(96,) DT det 13 False the (Hyderabad,) O 
(96,) NN pobj 11 False wallet (Hyderabad,) O 
(96,) . punct 9 False . (Hyderabad,) O 
(96,) NNP nsubj 16 True Hyderabad (Hyderabad,) B GPE
(96,) VBZ ROOT 16 False be (Hyderabad,) O 
(96,) DT det 18 False a (Hyderabad,) O 
(96,) NN attr 16 False city (Hyderabad,) O 
(96,) . punct 16 False . (Hyderabad,) O 
(96,) NNP nsubj 21 True Karthik (Hyderabad,) O 
(96,) VBZ ROOT 21 False be (Hyderabad,) O 
(96,) PRP$ poss 23 False my (Hyderabad,) O 
(96

In [338]:
vecs = [token.vector for token in doc]
np.dot(vecs[0], vecs[1]) / (np.dot(vecs[0], vecs[0]) + np.dot(vecs[1], vecs[1]))

0.119121194

In [368]:
for key in nlp.analyze_pipes().keys():
    print('key_0:', key)
    for key_ in nlp.analyze_pipes()[key].keys():
        print('\t', 'key_1:', key_)
        print('\t\t', nlp.analyze_pipes()[key][key_])

key_0: summary
	 key_1: tok2vec
		 {'assigns': ['doc.tensor'], 'requires': [], 'scores': [], 'retokenizes': False}
	 key_1: tagger
		 {'assigns': ['token.tag'], 'requires': [], 'scores': ['tag_acc'], 'retokenizes': False}
	 key_1: parser
		 {'assigns': ['token.dep', 'token.head', 'token.is_sent_start', 'doc.sents'], 'requires': [], 'scores': ['dep_uas', 'dep_las', 'dep_las_per_type', 'sents_p', 'sents_r', 'sents_f'], 'retokenizes': False}
	 key_1: attribute_ruler
		 {'assigns': [], 'requires': [], 'scores': [], 'retokenizes': False}
	 key_1: lemmatizer
		 {'assigns': ['token.lemma'], 'requires': [], 'scores': ['lemma_acc'], 'retokenizes': False}
	 key_1: ner
		 {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'], 'requires': [], 'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'], 'retokenizes': False}
key_0: problems
	 key_1: tok2vec
		 []
	 key_1: tagger
		 []
	 key_1: parser
		 []
	 key_1: attribute_ruler
		 []
	 key_1: lemmatizer
		 []
	 key_1: ner
		 []
key_0: attrs


In [364]:
list_ = dir(spacy.attrs)[:-12]
list_.remove('IDS')
list_.remove('NAMES')
doc_gram_feats = doc.to_array(list_)

In [132]:
import numpy as np
meaning_matrix = np.zeros((len(doc), len(doc)))
for i in range(len(doc)):
    for j in range(len(doc)):
        meaning_matrix[i, j] = np.mean((doc_gram_feats[i] == doc_gram_feats[j])*1)

In [133]:
np.min(meaning_matrix)

0.7906976744186046

In [134]:
doc

the bike is a bike

In [135]:
doc[-2]

a

In [136]:
words = [word for word in doc]

In [137]:
words[-1].has_vector

True

In [138]:
meaning_matrix

array([[1.        , 0.81395349, 0.81395349, 0.86046512, 0.79069767],
       [0.81395349, 1.        , 0.81395349, 0.8255814 , 0.95348837],
       [0.81395349, 0.81395349, 1.        , 0.8255814 , 0.80232558],
       [0.86046512, 0.8255814 , 0.8255814 , 1.        , 0.80232558],
       [0.79069767, 0.95348837, 0.80232558, 0.80232558, 1.        ]])